# Tutorial para principiantes

## Presentación del problema

(Aquí vendrá en enunciado)


## Formulación

### Conjuntos

$P$: plantas \\
$C$: clientes

### Parámetros
$O_p$: oferta de la planta $p \in P$ \\
$D_p$: demanda del cliente $c \in C$ \\
$C_{pc}$: coste por enviar un camión de la planta $p \in P$ al cliente $c \in C$

### Variables
$z$: coste total \\
$x_{pc}$: número de camiones enviados de la planta $p \in P$ al cliente $c \in C$

### Restricciones

$\sum_{p \in P} x_{pc} \leq O_p$, $\forall p \in P$ (Capacidad)


$\sum_{c \in C} x_{pc} \geq D_c$, $\forall c \in C$ (Demanda)


$z =\sum_{p \in P} \sum_{c \in C}C_{pc}x_{pc}$ (Coste total)

$x_{pc} \geq 0 \, \forall p, c$ (no negatividad)

### Función objetivo
min. $z$

## Datos de entrada
### Importando archivo de Excel

In [21]:
#!pip install -q xlrd
import xlrd
from google.colab import files

uploaded = files.upload()

ruta ='datos_entrada.xlsx'
wb = xlrd.open_workbook(ruta)

sheet = wb.sheet_by_name("oferta")
Oferta = {sheet.cell_value(r,0): sheet.cell_value(r,1)
          for r in range(0, sheet.nrows)}
Plantas = list(Oferta.keys())

sheet = wb.sheet_by_name("demanda")
Demanda = {sheet.cell_value(r,0): sheet.cell_value(r,1)
          for r in range(0, sheet.nrows)}
Clientes = list(Demanda.keys())

sheet = wb.sheet_by_name("coste_transporte")
CosteUnitario = {(sheet.cell_value(r,0), sheet.cell_value(r,1)): sheet.cell_value(r,2)
          for r in range(0, sheet.nrows)}

print (Plantas)

['Caceres', 'Zamora']


In [59]:
print(Plantas)
print (Clientes)
print (Oferta)
print (Demanda)
print (CosteUnitario)

['Caceres', 'Zamora']
['Madrid', 'Tarragona', 'Santander', 'Albacete', 'Sevilla']
{'Caceres': 47.0, 'Zamora': 63.0}
{'Madrid': 31.0, 'Tarragona': 16.0, 'Santander': 22.0, 'Albacete': 12.0, 'Sevilla': 28.0}
{('Caceres', 'Madrid'): 297.0, ('Caceres', 'Tarragona'): 831.0, ('Caceres', 'Santander'): 573.0, ('Caceres', 'Albacete'): 504.0, ('Caceres', 'Sevilla'): 264.0, ('Zamora', 'Madrid'): 248.0, ('Zamora', 'Tarragona'): 694.0, ('Zamora', 'Santander'): 344.0, ('Zamora', 'Albacete'): 499.0, ('Zamora', 'Sevilla'): 536.0}


### Introduciendo los datos manualmente

In [0]:
Plantas = ['Caceres', 'Zamora']
Clientes = ['Madrid', 'Tarragona', 'Santander', 'Albacete', 'Sevilla']

Oferta = {'Caceres':47, 'Zamora':63}
Demanda = {'Madrid':31, 'Tarragona':16, 
           'Santander':22, 'Albacete':12, 'Sevilla':28}



## Modelo matemático en PuLP

In [0]:
pip install pulp

     |████████████████████████████████| 40.6MB 111kB/s 


In [0]:
from pulp import*

In [0]:
modelMinCoste = LpProblem("MinCoste", LpMinimize)

In [0]:
Transporte=LpVariable.dicts('Transporte', [(p,c) for p in Plantas for c in Clientes], lowBound = 0)

In [0]:
for p in Plantas:
    modelMinCoste+=LpConstraint(sum(Transporte[p,c] for c in Clientes), sense=LpConstraintGE, rhs= Oferta[p], name='Oferta'+str(p))

In [0]:
for c in Clientes:
    modelMinCoste+=LpConstraint(sum(Transporte[p,c] for p in Plantas), sense=LpConstraintGE, rhs= Demanda[c], name='Demanda'+str(c))

In [0]:
modelMinCoste+=sum(Transporte[p,c] for p in Plantas for c in Clientes)

In [0]:
modelMinCoste.solve()

1

## Plan de transporte óptimo

In [0]:
  print ('Coste mínimo: ' + str(value(modelMinCoste.objective)))
  for p in Plantas:
      for c in Clientes:
          if Transporte[p,c].varValue>0:
               print (str(Transporte[p,c].varValue) + ' camiones de ' + str(p) + ' a ' +str(c))


Coste mínimo: 110.0
19.0 camiones de Caceres a Madrid
28.0 camiones de Caceres a Sevilla
13.0 camiones de Zamora a Madrid
16.0 camiones de Zamora a Tarragona
22.0 camiones de Zamora a Santander
12.0 camiones de Zamora a Albacete


## Modelo matemático en Pyomo

### Instalación de Pyomo

In [0]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass
from pyomo.environ import*

### Pyomo concrete model

In [0]:
m = ConcreteModel()

In [0]:
m.Transporte = Var(Plantas, Clientes, within=NonNegativeReals)

In [0]:
for p in Plantas:
  m.str("ResOferta%p", %p) = Constraint(expr = sum(m.Transporte[p, c] for c in Clientes)<= Oferta[p])

SyntaxError: ignored

In [0]:
for c in Clientes:
  m.ResDemanda = Constraint(expr = sum(m.Transporte[p, c] for p in Plantas)>= Demanda[c])

In [0]:
m.value = Objective (expr = sum(m.Transporte[p,c]*CosteUnitario[p,c] for p in Plantas for c in Clientes))

In [0]:
# Optimize
solver = SolverFactory('cbc')
status = solver.solve(m)

In [0]:

# Print the status of the solved LP
print("Status = %s" % status.solver.termination_condition)

# Print solution
print ("Coste total: %f" %value(m.value))
for (p,c) in [(p,c) for p in Plantas for c in Clientes if value(m.Transporte[p,c])>0]:
  print ("Transporte de %f camiones de %s a %s" %(value(m.Transporte[p,c]),p,c))

In [0]:
del(m)

### Pyomo abstract model

In [0]:
# Creating the model
mPyAbs = AbstractModel()

In [0]:
# Defining sets
mPyAbs.sPlants = Set()
mPyAbs.sCustomers = Set()

In [0]:
# Defining parameters
mPyAbs.pDemand = Param(mPyAbs.sCustomers, mutable=True)
mPyAbs.pSupply = Param(mPyAbs.sPlants, mutable=True)
mPyAbs.pUnitTransportCost = Param(mPyAbs.sPlants, mPyAbs.sCustomers, mutable=True)

In [0]:
#Defining variables
mPyAbs.vTransport = Var(mPyAbs.sPlants, mPyAbs.sCustomers, within=NonNegativeReals)
mPyAbs.vTotalCost = Var(within=NonNegativeReals)

In [0]:
# Defining supply constraint rule
def fcSupply (model, p):
  return sum(model.vTransport[p,c] for c in model.sCustomers) <= \
    model.pSupply[p]

In [0]:
# Defining demand constraint rule
def fcDemand (model, c):
  return sum(model.vTransport[p,c] for p in model.sPlants) >= \
    model.pDemand[c]

In [0]:
# Defining total cost rule
def fvTotalCost (model):
  return model.vTotalCost == sum (model.vTransport[p,c]*model.pUnitTransportCost[p,c]
                                  for p in model.sPlants
                                  for c in model.sCustomers)

In [0]:
# Defining objective function rule
def obj_expression(model):
  return model.vTotalCost

In [0]:
# Activating constraints
mPyAbs.cSupply = Constraint(mPyAbs.sPlants, rule=fcSupply)
mPyAbs.cDemand = Constraint(mPyAbs.sCustomers, rule=fcDemand)
mPyAbs.cTotalCost = Constraint(rule=fvTotalCost)

In [0]:
# Setting the objective
mPyAbs.fObj = Objective(rule=obj_expression)

In [0]:
# Preparing model input data
input_data = {None:{
    'sPlants': {None: Plantas},
    'sCustomers': {None:Clientes},
    'pSupply': Oferta,            
    'pDemand': Demanda,   
    'pUnitTransportCost': CosteUnitario,
}}

In [0]:
# Creating model instance
instance = mPyAbs.create_instance(input_data)

In [74]:
opt = SolverFactory('cbc')
results = opt.solve(instance)

40094.0
Transport %r from %s to %s 6.0 1 1
Transport %r from %s to %s 0.0 1 1
Transport %r from %s to %s 0.0 1 1
Transport %r from %s to %s 12.0 1 1
Transport %r from %s to %s 28.0 1 1
Transport %r from %s to %s 25.0 1 1
Transport %r from %s to %s 16.0 1 1
Transport %r from %s to %s 22.0 1 1
Transport %r from %s to %s 0.0 1 1
Transport %r from %s to %s 0.0 1 1


In [80]:
print("Total cost: " + str(instance.vTotalCost.value))
for (p, c) in [i for i in instance.vTransport.keys()]:
  print("Transport " + str(instance.vTransport[p,c].value) + " from " + p + " to " + c)

Total cost: 40094.0
Transport 6.0 from Caceres to Madrid
Transport 0.0 from Caceres to Tarragona
Transport 0.0 from Caceres to Santander
Transport 12.0 from Caceres to Albacete
Transport 28.0 from Caceres to Sevilla
Transport 25.0 from Zamora to Madrid
Transport 16.0 from Zamora to Tarragona
Transport 22.0 from Zamora to Santander
Transport 0.0 from Zamora to Albacete
Transport 0.0 from Zamora to Sevilla
